# Optimizing Python Code with [Cython](https://cython.org/)

In [ ]:
import Cython
from random import random
import math
import numpy as np
import matplotlib.pyplot as plt

# This is needed to load the Cython magic
%load_ext Cython 

## Calculating $\pi$ using Monte Carlo Integration

### Pure Python

In [ ]:
def pi_mc(n=1000):
    '''Calculate PI using Monte Carlo method'''
    in_circle = 0
    for i in range(n):
        x, y = random(), random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc(10000000)

### Cython

#### 1. Use `cython` cell magic without making any code change

In [ ]:
%%cython

from random import random

def pi_mc(n=1000):
    '''Calculate PI using Monte Carlo method'''
    in_circle = 0
    for i in range(n):
        x, y = random(), random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc(10000000)

#### 2. Static type declarations in Cython with `cdef`

Static type declarations allow Cython to step out of the dynamic nature of the Python code and produce efficient **C** code.

In [ ]:
%%cython

from random import random

def pi_mc(n=1000):
    '''Calculate PI using Monte Carlo method'''
    cdef int in_circle = 0
    cdef int i
    cdef double x, y
    for i in range(n):
        x, y = random(), random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc(10000000)

#### 3. Using Cython `annotate` option and inspect the generated C code

In [ ]:
%%cython --annotate

from random import random

def pi_mc(int n=1000):
    '''Calculate PI using Monte Carlo method'''
    cdef:
        int in_circle = 0
        int i
        double x, y
    for i in range(n):
        x, y = random(), random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

In [ ]:
%time pi_mc(10000000)

## Types of Cython functions

Cython offers three different types of function declared with `def`, `cdef`, `cpdef`:

1. Functions declared with `def` can be called from Python and Cython code.
2. Functions declared with `cdef` can be only called from Cython code.
3. `cpdef` causes Cython to create two versions of the function. One which is used when the function is called from Cython code and one when it is called from Python.

In [ ]:
%%cython 


cdef double cube(double x):
    return x * x * x


cpdef double cube_sum(double x, double y):
    return cube(x) + cube(y)

### Function Inlining

We can use the `cdef inline` for small functions that are used often and Cython inlines them reducing overhead

In [ ]:
%%cython --annotate


cdef inline double cube(double x):
    return x * x * x


cpdef double cube_sum(double x, double y):
    return cube(x) + cube(y)

## Typed Memory Views

Cython allows access to the contents of NumPy arrays by **memory views**.

In [ ]:
%%cython --annotate

def my_sum(double[:] x):
    cdef int i, nx = x.shape[0]
    cdef double s = 0.0
    for i in range(nx):
        s += x[i]
        
    return s

In [ ]:
x = np.ones(1000, dtype=np.float64)
s1 = my_sum(x)
s2 = x.sum()
print(s1, s2)

In [ ]:
%%cython --annotate

def my_sum2d(double[:, :] x):
    cdef int i, j, nx = x.shape[0], ny = x.shape[1]
    cdef double s = 0.0
    for i in range(nx):
        for j in range(ny):
            s += x[i, j]
        
    return s

In [95]:
x = np.ones((1000, 1000), dtype=np.float64)
s1 = my_sum2d(x)
s2 = x.sum()
print(s1, s2)

1000000.0 1000000.0


## Disable bounds checking and wraparound

For safety reasons, Cython checks if we try to access elements out of the array boundaries. 
Furthermore it allows using negative array indices. We can exhange safety with performance disabling both of them.


In [ ]:
%%cython --annotate

from cython cimport wraparound, boundscheck

@wraparound(False)
@boundscheck(False)
cpdef double my_sum2d(double[:, :] x):
    cdef int i, j, nx = x.shape[0], ny = x.shape[1]
    cdef double s = 0.0
    for i in range(nx):
        for j in range(ny):
            s += x[i, j]
        
    return s

## Releasing the GIL and parallelizing loops

### Calculating Julia Sets

In [ ]:
%%cython

from cython cimport boundscheck, wraparound

@wraparound(False)
@boundscheck(False)
def julia_set_cython(const double[:, :] X, const double[:, :] Y,
                     const double cx, const double cy,
                     const int iter_max, const double radius2, 
                     int[:, :] julia):
    cdef:
        int i, j, k, nx, ny
        double x, y
    nx = X.shape[0]
    ny = Y.shape[1]
    for i in range(nx):
        for j in range(ny):
            x = X[i, j]
            y = Y[i, j]
            k = 0
            while x * x + y * y < radius2 and k < iter_max:
                x, y = x * x - y * y + cx, 2.0 * x * y + cy
                k = k + 1
                
            julia[i, j] = k

In [ ]:
X, Y = np.meshgrid(np.linspace(-2.0 , 2.0, 5000), np.linspace(-2.0, 2.0, 5000))
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
julia = np.zeros_like(X, dtype=np.int32)
c = -0.9 + 0.22143j
radius2 = 4.0
%timeit julia_set_cython(X, Y, c.real, c.imag, 100, radius2, julia)
ax.set_aspect('equal')
ax.imshow(julia, extent=[-2, 2, -2, 2]);

In [ ]:
%%cython

from cython cimport boundscheck, wraparound
from cython.parallel cimport prange

@boundscheck(False)
@wraparound(False)
def julia_set_cython(const double[:, :] X, const double[:, :] Y,
                     const double cx, const double cy,
                     const int iter_max, const double radius2, 
                     int[:, :] julia):
    cdef:
        int i, j, k, nx, ny
        double x, y
    nx = X.shape[0]
    ny = Y.shape[1]
    for i in prange(nx, nogil=True):
        for j in range(ny):
            x = X[i, j]
            y = Y[i, j]
            k = 0
            while x * x + y * y < radius2 and k < iter_max:
                x, y = x * x - y * y + cx, 2.0 * x * y + cy
                k = k + 1
                
            julia[i, j] = k

In [ ]:
X, Y = np.meshgrid(np.linspace(-2.0 , 2.0, 5000), np.linspace(-2.0, 2.0, 5000))
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
julia = np.zeros_like(X, dtype=np.int32)
c = -0.9 + 0.22143j
radius2 = 4.0
%timeit julia_set_cython(X, Y, c.real, c.imag, 100, radius2, julia)
ax.set_aspect('equal')
ax.imshow(julia, extent=[-2, 2, -2, 2]);

In [ ]:
%%cython

from cython cimport boundscheck, wraparound
from cython.parallel cimport prange

@boundscheck(False)
@wraparound(False)
def julia_set_cython(const double[:, :] X, const double[:, :] Y,
                     const double cx, const double cy,
                     const int iter_max, const double radius2, 
                     int[:, :] julia):
    cdef:
        int i, j, k, nx, ny
        double x, y
    nx = X.shape[0]
    ny = Y.shape[1]
    for i in prange(nx, nogil=True):
        for j in range(ny):
            x = X[i, j]
            y = Y[i, j]
            k = 0
            while x * x + y * y < radius2 and k < iter_max:
                x, y = x * x - y * y + cx, 2.0 * x * y + cy
                k = k + 1
                
            julia[i, j] = k

In [ ]:
X, Y = np.meshgrid(np.linspace(-2.0 , 2.0, 5000), np.linspace(-2.0, 2.0, 5000))
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
julia = np.zeros_like(X, dtype=np.int32)
c = -0.9 + 0.22143j
radius2 = 4.0
%timeit julia_set_cython(X, Y, c.real, c.imag, 100, radius2, julia)
ax.set_aspect('equal')
ax.imshow(julia, extent=[-2, 2, -2, 2]);

### !!! WE FORGOT TO PUT THE CORRECT COMPILATION/LINKING OPTIONS !!!

In [ ]:
%%cython

# distutils: extra_compile_args = -fopenmp -march=native
# distutils: extra_link_args = -fopenmp
from cython cimport boundscheck, wraparound
from cython.parallel cimport prange

@boundscheck(False)
@wraparound(False)
def julia_set_cython(const double[:, :] X, const double[:, :] Y,
                     const double cx, const double cy,
                     const int iter_max, const double radius2, 
                     int[:, :] julia):
    cdef:
        int i, j, k, nx, ny
        double x, y
    nx = X.shape[0]
    ny = Y.shape[1]
    for i in prange(nx, nogil=True):
        for j in range(ny):
            x = X[i, j]
            y = Y[i, j]
            k = 0
            while x * x + y * y < radius2 and k < iter_max:
                x, y = x * x - y * y + cx, 2.0 * x * y + cy
                k = k + 1
                
            julia[i, j] = k

In [ ]:
X, Y = np.meshgrid(np.linspace(-2.0 , 2.0, 5000), np.linspace(-2.0, 2.0, 5000))
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111)
julia = np.zeros_like(X, dtype=np.int32)
c = -0.9 + 0.22143j
radius2 = 4.0
%timeit julia_set_cython(X, Y, c.real, c.imag, 100, radius2, julia)
ax.set_aspect('equal')
ax.imshow(julia, extent=[-2, 2, -2, 2]);